In [1]:
from OurTrainingTools2D import *

=========== Random Seed: 650 ===========


In [2]:
class OurCDModel(nn.Module):
### Defines the  model with parametrized discriminant. Only quadratic dependence on a single parameter is implemented.
### Input is the architecture (list of integers, the last one being equal to 1) and the activation type ('ReLU' or 'Sigmoid')
    def __init__(self, NumberOfParameters, AR = [1, 3, 3, 1] , AF = 'ReLU'):               
        super(OurCDModel, self).__init__() 
        ValidActivationFunctions = {'ReLU': torch.relu, 'Sigmoid': torch.sigmoid}
        try:
            self.ActivationFunction = ValidActivationFunctions[AF]
        except KeyError:
            print('The activation function specified is not valid. Allowed activations are %s.'
                 %str(list(ValidActivationFunctions.keys())))
            print('Will use ReLU.')
            self.ActivationFunction = torch.relu            
        if type(AR) == list:
            if( ( all(isinstance(n, int) for n in AR)) and ( AR[-1] == 1) ):
                self.Architecture = AR
            else:
                print('Architecture should be a list of integers, the last one should be 1.')
                raise ValueError             
        else:
            print('Architecture should be a list !')
            raise ValueError
        self.NumberOfParameters = NumberOfParameters

### Define Layers
        self.NumberOfNetworks = int((2+NumberOfParameters)*(1+NumberOfParameters)/2)-1
        LinearLayers = [([nn.Linear(self.Architecture[i], self.Architecture[i+1]) \
                                  for i in range(len(self.Architecture)-1)])\
                        for n in range(self.NumberOfNetworks)]
        LinearLayers = [Layer for SubLayerList in LinearLayers for Layer in SubLayerList]
        self.LinearLayers = nn.ModuleList(LinearLayers)
        
    def Forward(self, Data, Parameters):
### Forward Function. Performs Preprocessing, returns F = rho/(1+rho) in [0,1], where rho is quadratically parametrized.
        # Checking that data has the right input dimension
        InputDimension = self.Architecture[0]
        if Data.size(1) != InputDimension:
            print('Dimensions of the data and the network input mismatch: data: %d, model: %d'
                  %(Data.size(1), InputDimension))
            raise ValueError

        # Checking that preprocess has been initialised
        if not hasattr(self, 'Shift'):
            print('Please initialize preprocess parameters!')
            raise ValueError
        with torch.no_grad(): 
            Data, Parameters = self.Preprocess(Data, Parameters)  
        
        NumberOfLayers, NumberOfEvents = len(self.Architecture)-1, Data.size(0)
        EntryIterator, NetworkIterator = 0, -1
        MatrixLT = torch.zeros([NumberOfEvents, (self.NumberOfParameters+1)**2], dtype=Data.dtype)
        
        if Data.is_cuda:
            MatrixLT = OurCudaTensor(MatrixLT)
        
        for i in range(self.NumberOfParameters+1):
            EntryIterator += i
            DiagonalEntry = True
            for j in range(self.NumberOfParameters+1-i):
                if NetworkIterator == -1:
                    MatrixLT[:, EntryIterator] = torch.ones(NumberOfEvents)
                    #print('Entry: %d, Layer: ones, DiagonalEntry: %s'%(EntryIterator,
                    #                                                str(DiagonalEntry)))
                else:
                    x = Data
                    for Layer in self.LinearLayers[NumberOfLayers*NetworkIterator:\
                                                  NumberOfLayers*(NetworkIterator+1)-1]:
                        x = self.ActivationFunction(Layer(x))
                    x = self.LinearLayers[NumberOfLayers*(NetworkIterator+1)-1](x).squeeze()
                    #MatrixLT[:, EntryIterator] = torch.exp(x) if DiagonalEntry else x
                    MatrixLT[:, EntryIterator] = x
                    #print('Entry: %d, Layer: %d, DiagonalEntry: %s'%(EntryIterator, NetworkIterator, 
                    #                                                str(DiagonalEntry)))
                EntryIterator += 1
                NetworkIterator += 1
                DiagonalEntry = False
        #print('MatrixLT: '+str(MatrixLT.is_cuda))
        #print('Parameters: '+str(Parameters.is_cuda))

        MatrixLT = MatrixLT.reshape([-1, self.NumberOfParameters+1, self.NumberOfParameters+1])
        MatrixLTP = MatrixLT.matmul(Parameters.reshape([NumberOfEvents, self.NumberOfParameters+1, 1]))
        rho = MatrixLTP.permute([0, 2, 1]).matmul(MatrixLTP).squeeze()
        
        return (rho.div(1.+rho)).view(-1, 1)
    
    def GetL1Bound(self, L1perUnit):
        self.L1perUnit = L1perUnit
    
    def ClipL1Norm(self):
### Clip the weights      
        def ClipL1NormLayer(DesignatedL1Max, Layer, Counter):
            if Counter == 1:
                ### this avoids clipping the first layer
                return
            L1 = Layer.weight.abs().sum()
            Layer.weight.masked_scatter_(L1 > DesignatedL1Max, 
                                        Layer.weight*(DesignatedL1Max/L1))
            return
        
        Counter = 0
        for m in self.children():
            if isinstance(m, nn.Linear):
                Counter += 1
                with torch.no_grad():
                    DesignatedL1Max = m.weight.size(0)*m.weight.size(1)*self.L1perUnit
                    ClipL1NormLayer(DesignatedL1Max, m, Counter)
            else:
                for mm in m:
                    Counter +=1
                    with torch.no_grad():
                        DesignatedL1Max = mm.weight.size(0)*m.weight.size(1)*self.L1perUnit
                        ClipL1NormLayer(DesignatedL1Max, mm, Counter)
        return 
    
    def DistributionRatio(self, points):
### This is rho. I.e., after training, the estimator of the distribution ratio.
        with torch.no_grad():
            F = self(points)
        return F/(1-F)

    def InitPreprocess(self, Data, Parameters):
### This can be run only ONCE to initialize the preprocess (shift and scaling) parameters
### Takes as input the training Data and the training Parameters as Torch tensors.
        if not hasattr(self, 'Scaling'):
            print('Initializing Preprocesses Variables')
            self.Scaling = Data.std(0)
            self.Shift = Data.mean(0)
            self.ParameterScaling = Parameters.std(0)  
        else: print('Preprocess can be initialized only once. Parameters unchanged.')
            
    def Preprocess(self, Data, Parameters):
### Returns scaled/shifted data and parameters
### Takes as input Data and Parameters as Torch tensors.
        if  not hasattr(self, 'Scaling'): print('Preprocess parameters are not initialized.')
        Data = (Data - self.Shift)/self.Scaling
        Parameters = Parameters/self.ParameterScaling
        Ones = torch.ones([Parameters.size(0),1], dtype=Parameters.dtype)
        if Parameters.is_cuda:
            Ones = Ones.cuda()
        Parameters = torch.cat([Ones, Parameters.reshape(Data.size(0), -1)], dim=1)
        return Data, Parameters
    
    def Save(self, Name, Folder, csvFormat=False):
### Saves the model in Folder/Name
        FileName = Folder + Name + '.pth'
        torch.save({'StateDict': self.state_dict(), 
                   'Scaling': self.Scaling,
                   'Shift': self.Shift,
                   'ParameterScaling': self.ParameterScaling}, 
                   FileName)
        print('Model successfully saved.')
        print('Path: %s'%str(FileName))
        
        if csvFormat:
            modelparams = [w.detach().tolist() for w in self.parameters()]
            np.savetxt(Folder + Name + ' (StateDict).csv', modelparams, '%s')
            statistics = [self.Shift.detach().tolist(), self.Scaling.detach().tolist(),
                         self.ParameterScaling.detach().tolist()]
            np.savetxt(Folder + Name + ' (Statistics).csv', statistics, '%s')
    
    def Load(self, Name, Folder):
### Loads the model from Folder/Name
        FileName = Folder + Name + '.pth'
        try:
            IncompatibleKeys = self.load_state_dict(torch.load(FileName)['StateDict'])
        except KeyError:
            print('No state dictionary saved. Loading model failed.')
            return 
        
        if list(IncompatibleKeys)[0]:
            print('Missing Keys: %s'%str(list(IncompatibleKeys)[0]))
            print('Loading model failed. ')
            return 
        
        if list(IncompatibleKeys)[1]:
            print('Unexpected Keys: %s'%str(list(IncompatibleKeys)[0]))
            print('Loading model failed. ')
            return 
        
        self.Scaling = torch.load(FileName)['Scaling']
        self.Shift = torch.load(FileName)['Shift']
        self.ParameterScaling = torch.load(FileName)['ParameterScaling']
        
        print('Model successfully loaded.')
        print('Path: %s'%str(FileName))
        
    def Report(self): ### is it possibe to check if the model is in double?
        print('\nModel Report:')
        print('Preprocess Initialized: ' + str(hasattr(self, 'Shift')))
        print('Architecture: ' + str(self.Architecture))
        print('Loss Function: ' + 'Quadratic')
        print('Activation: ' + str(self.ActivationFunction))
        
    def cuda(self):
        nn.Module.cuda(self)
        self.Shift = self.Shift.cuda()
        self.Scaling = self.Scaling.cuda()
        self.ParameterScaling = self.ParameterScaling.cuda()
        
    def cpu(self):
        self.Shift = self.Shift.cpu()
        self.Scaling = self.Scaling.cpu()
        self.ParameterScaling = self.ParameterScaling.cpu()
        return nn.Module.cpu(self)

    



In [3]:
class OurTrainingData():
### Imports data for training. The Return() methods returns [self.Data, self.Labels, self.Weights, self.ParVal]
### All values are in double precision
### Inputs are the SM and BSM file paths and list of integers to chop the datasets if needed
### Weights are normalized to have sum = 1 on the entire training sample
    def __init__(self, SMfilepathlist, BSMfilepathlist, process, parameters, SMNLimits="NA", BSMNLimits="NA", verbose=True): 
        self.Process = process
        self.Parameters = parameters
        if verbose: print('Loading Data Files for Process: ' + str(self.Process) +', with new physics Parameters: ' + str(self.Parameters) ) 
        if len(self.Parameters)!= 1: print('Only 1D Implemented in Training !')   
          
####### Load BSM data (stored in self.BSMDataFiles)
        if type(BSMfilepathlist) == list:
            if all(isinstance(n, str) for n in BSMfilepathlist):
                self.BSMDataFiles = [] 
                for path in BSMfilepathlist:
                    temp =  DataFile(path, verbose=verbose)
                    if((temp.Process == self.Process) and (set(list(temp.Parameters.flatten())) == set(self.Parameters)) and (sum(temp.Values.flatten()) != 0.) ):
                        self.BSMDataFiles.append(temp)
                    else: 
                        print('File not valid: ' + path)
                        print('Parameters = ' + str(temp.Parameters) + ', Process = ' + str(temp.Process) 
                              +' and Values = ' + str(temp.Values.tolist()))
                        print('should be = ' + str(self.Parameters) + ', = ' + str(self.Process) 
                              + ' and != ' + str(0.))
                        raise ValueError
                        self.BSMDataFiles.append(None) 
            else:
                print('BSMfilepathlist input should be a list of strings !')
                raise FileNotFoundError
        else:
            print('BSMfilepathlist input should be a list !')
            raise FileNotFoundError
                  
###### Chop the BSM data sets (stored in BSMNDList, BSMDataList, BSMWeightsList, BSMParValList, BSMTargetList)
        if type(BSMNLimits) == int:
            BSMNLimits = [min(BSMNLimits, NF.ND) for NF in self.BSMDataFiles]
        elif type(BSMNLimits) == list and all(isinstance(n, int) for n in BSMNLimits):
            if len(BSMNLimits) != len(self.BSMDataFiles):
                print("--> Please input %d integers to chop each SM file."%(
                    len(self.BSMDataFiles)))
                raise ValueError
            elif sum([self.BSMDataFiles[i].ND >= BSMNLimits[i] for i in range(len(BSMNLimits))]
                    ) != len(self.BSMDataFiles):
                print("--> Some chop limit larger than available data in the corresponding file.")
                print("--> Lengths of the files: "+str([file.ND for file in self.BSMDataFiles ]))
                raise ValueError
        else:
            BSMNLimits =[file.ND for file in self.BSMDataFiles]   
            
        self.BSMNDList = BSMNLimits
        #self.BSMNData = sum(self.BSMNDataList)
        self.BSMDataList = [DF.Data[:N] for (DF, N) in zip(
            self.BSMDataFiles, self.BSMNDList)]
        self.BSMWeightsList = [DF.Weights[:N] for (DF, N) in zip(
            self.BSMDataFiles, self.BSMNDList)] 
        self.BSMXSList = [DF.XS for DF in self.BSMDataFiles]
        self.BSMParValList =  [torch.ones([N, len(self.Parameters)], dtype=torch.double)*DF.Values for (DF, N) in zip(self.BSMDataFiles, self.BSMNDList)]
        self.BSMTargetList = [torch.ones(N, dtype=torch.double) for N in self.BSMNDList] 
        
        
####### Load SM data (stored in SMDataFiles)
        if type(SMfilepathlist) == list:
            if all(isinstance(n, str) for n in SMfilepathlist):
                #self.SMFilePathList = SMfilepathlist
                #self.SMNumFiles = len(self.SMFilePathList)
                self.SMDataFiles = []
                for path in SMfilepathlist:
                    temp =  DataFile(path, verbose=verbose)
                    if( (temp.Process == self.Process) and (temp.Parameters[0] == 'SM') and (sum(temp.Values.flatten()) == 0.) ):
                        self.SMDataFiles.append(temp)
                    else:
                        print('File not valid: ' + path)
                        print('Parameters = ' + str(temp.Parameters) + ', Process = ' + str(temp.Process) 
                              +' and Values = ' + str(temp.Values.tolist()))
                        print('should be = ' + 'SM'+ ', = ' + str(self.Process) 
                              + ' and = ' + str(0.))
                        self.SMDataFiles.append(None)                    
            else:
                print('SMfilepathlist input should be a list of strings !')
                raise FileNotFoundError
        else:
            print('SMfilepathlist input should be a list !')
            raise FileNotFoundError
            
####### Chop the SM data sets and join them in one (stored in SMND, SMData and SMWeights)
        if type(SMNLimits) == int:
            SMNLimits = [min(SMNLimits, DF.ND) for DF in self.SMDataFiles]
        elif type(SMNLimits) == list and all(isinstance(n, int) for n in SMNLimits):
            if len(SMNLimits) != len(self.SMDataFiles):
                print("--> Please input %d integers to chop each SM file."%(
                    len(self.SMDataFiles)))
                raise ValueError
            elif sum([self.SMDataFiles[i].ND >= SMNLimits[i] for i in range(len(SMNLimits))]
                    ) != len(self.SMDataFiles):
                print("--> Some chop limit larger than available data in the corresponding file.")
                print("--> Lengths of the files: " + str([file.ND for file in self.SMDataFiles]))
                raise ValueError
        else:
            SMNLimits = [file.ND for file in self.SMDataFiles]
        self.SMND = sum(SMNLimits)
        self.SMData = torch.cat(
            [DF.Data[:N] for (DF, N) in zip(self.SMDataFiles, SMNLimits)]
            , 0) 
        self.SMWeights = torch.cat(
            [DF.Weights[:N] for (DF, N) in zip(self.SMDataFiles, SMNLimits)]
            , 0)
        self.SMXSList = [DF.XS for DF in self.SMDataFiles]
        
        
        #idx_random = torch.randperm(self.SMND)
        #self.SMData = self.SMData[idx_random, :]
        #self.SMWeights = self.SMWeights[idx_random]

####### Break SM data in blocks to be paired with BSM data (stored in UsedSMNDList, UsedSMDataList, UsedSMWeightsList, UsedSMParValList, UsedSMTargetList)
        BSMNRatioDataList = [torch.tensor(1., dtype=torch.double)*n/sum(self.BSMNDList
                                                                       ) for n in self.BSMNDList]
        self.UsedSMNDList = [int(self.SMND*BSMNRatioData) for BSMNRatioData in BSMNRatioDataList] 
        #self.UsedSMNData = sum(self.UsedSMNDataList)
        #self.UsedSMData = self.SMData[: self.UsedSMND]
        self.UsedSMDataList =  self.SMData[:sum(self.UsedSMNDList)].split(self.UsedSMNDList)
        
    ##### Reweighting is performed such that the SUM of the SM weights in each block equals the number of BSM data times the AVERAGE 
    ##### of the original weights. This equals the SM cross-section as obtained in the specific sample at hand, times NBSM
        self.UsedSMWeightsList = self.SMWeights[:sum(self.UsedSMNDList)].split(self.UsedSMNDList)
        self.UsedSMWeightsList = [ self.UsedSMWeightsList[i]*self.BSMNDList[i]/self.UsedSMNDList[i] for i in range(len(BSMNRatioDataList))]   
        self.UsedSMParValList =  [torch.ones([N, len(self.Parameters)], dtype=torch.double)*DF.Values for (DF, N) in zip(self.BSMDataFiles, self.UsedSMNDList)]       
        self.UsedSMTargetList = [torch.zeros(N, dtype=torch.double) for N in self.UsedSMNDList]

####### Join SM with BSM data
        self.Data = torch.cat(
            [torch.cat([self.UsedSMDataList[i], self.BSMDataList[i]]
                                  ) for i in range(len(self.BSMDataList))]
            )
        self.Weights = torch.cat(
            [torch.cat([self.UsedSMWeightsList[i], self.BSMWeightsList[i]]
                                  ) for i in range(len(self.BSMWeightsList))]
            )
        self.Labels = torch.cat(
            [torch.cat([self.UsedSMTargetList[i], self.BSMTargetList[i]]
                                  ) for i in range(len(self.BSMTargetList))]
            )
        self.ParVal = torch.cat(
            [torch.cat([self.UsedSMParValList[i], self.BSMParValList[i]]
                                  ) for i in range(len(self.BSMParValList))]
            )
        
####### Final reweighting
        s = self.Weights.sum()
        self.Weights = self.Weights.div(s)

####### If verbose, display report
        if verbose: self.Report()
        
####### Return Tranining Data
    def ReturnData(self):
        return [self.Data, self.Labels, self.Weights, self.ParVal]
            
    def Report(self):
        #from tabulate import tabulate
        print('\nLoaded SM Files:')
        print(tabulate({str(self.Parameters): [ file.Values for file in self.SMDataFiles ], 
                        "#Data":[ file.ND for file in self.SMDataFiles ], 
                        "XS[pb](avg.w)":[ file.XS for file in self.SMDataFiles ]}, headers="keys"))
        print('\nLoaded BSM Files:')
        print(tabulate({str(self.Parameters): [ file.Values for file in self.BSMDataFiles ], 
                        "#Data":[ file.ND for file in self.BSMDataFiles ], 
                        "XS[pb](avg.w)":[ file.XS for file in self.BSMDataFiles ]}, headers="keys"))
        print('\nPaired BSM/SM Datasets:\n')
        ### Check should be nearly equal to #EV.BSM. It is computed with the weights BEFORE final reweighting
        print(tabulate({str(self.Parameters): [ file.Values for file in self.BSMDataFiles ], "#Ev.BSM": self.BSMNDList
                        , "#Ev.SM": self.UsedSMNDList,
                        "Check": [(self.UsedSMWeightsList[i].sum())/(self.SMWeights.mean()) for i in range(len(self.BSMDataFiles))]
                       }, headers="keys"))    
        
####### Convert Angles
    def CurateAngles(self, AnglePos):
        Angles = self.Data[:, AnglePos]
        CuratedAngles = torch.cat([torch.sin(Angles), torch.cos(Angles)], dim=1)
        OtherPos = list(set(range(self.Data.size(1)))-set(AnglePos))
        self.Data = torch.cat([self.Data[:, OtherPos], CuratedAngles], dim=1)
        print('####\nAnlges at position %s have been converted to Sin and Cos and put at the last columns of the Data.'%(AnglePos))
        print('####')

## New Data Format

erased SM randomisation

## unbalanced SM/BSM ratio

In [4]:
DataFolder = '/data3/WZ_new_project/h5/Ideal_Data'

td = OurTrainingData([DataFolder + '/ChP_pt300_sm_1.h5',],
                    [DataFolder + '/ChP_pt300_gphi2e-1_gw1e-1.h5',
                     DataFolder + '/ChP_pt300_gphi5e-1_gw2e-1.h5',
                     DataFolder + '/ChP_pt300_gphim2e-1_gw1e-1.h5',
                     DataFolder + '/ChP_pt300_gphim5e-1_gw2e-1.h5',
                     DataFolder + '/ChP_pt300_gphim5e-1.h5',
                     DataFolder + '/ChP_pt300_gphim2e-1.h5',
                     DataFolder + '/ChP_pt300_gphim5e-2.h5',
                     DataFolder + '/ChP_pt300_gphi5e-1.h5',
                     DataFolder + '/ChP_pt300_gphi2e-1.h5',
                     DataFolder + '/ChP_pt300_gphi5e-2.h5',
                     DataFolder + '/ChP_pt300_gwm5e-2.h5',
                     DataFolder + '/ChP_pt300_gwm1e-1.h5',
                     DataFolder + '/ChP_pt300_gwm2e-1.h5',                     
                     DataFolder + '/ChP_pt300_gw5e-2.h5',
                     DataFolder + '/ChP_pt300_gw1e-1.h5',
                     DataFolder + '/ChP_pt300_gw2e-1.h5'],
                     process = 'W+Z', parameters =['Gphi[TeV**-2]', 'GW[TeV**-2]'], 
                     SMNLimits=int(3e6),
                     BSMNLimits=int(5e5))

Loading Data Files for Process: W+Z, with new physics Parameters: ['Gphi[TeV**-2]', 'GW[TeV**-2]']
Only 1D Implemented in Training !

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChP_pt300_gphi2e-1_gw1e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {0.2, 0.1}[TeV**-2] data, Ideal Events. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ}, weight}.
Converted from /data3/WZ_new_project/dat/Ideal_Events/ChP_pt300_gphi2e-1_gw1e-1.dat.gz
Charge = 1 --- Process = W+Z
#####

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChP_pt300_gphi5e-1_gw2e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {0.5, 0.2}[TeV**-2] data, Ideal Events. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ}, weight}.
Converted from /data3/WZ_new_project/dat/Ideal_Events/ChP_pt300_gphi5e-1_gw2e-1.dat.gz
Charge = 1 --- Process = W+Z
#####

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChP_pt300_gphim2e-1_gw1e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {-0.2, 0.1}[TeV**-2] data, Ideal

In [6]:
NumEpochs = int(1e4)

td.Data = td.Data[:, :7]
td.CurateAngles([3, 5])

Data, ParVal, Labels, Weights = td.Data, td.ParVal, td.Labels, td.Weights
Data, ParVal, Labels, Weights = Data.float(), ParVal.float(), Labels.float(), Weights.float()

MD = OurCDModel(NumberOfParameters=2, AR=[9,32,32,32,1])
MD.InitPreprocess(Data, ParVal)

OT = OurTrainer(NumEpochs = NumEpochs)
OT.SetSaveAfterEpochs([10,100,500]+list(range(1000, 11000, 1000)))

random_seed = torch.randint(0, 1339, (1,)).item()
OT.Train(MD, Data = Data, Parameters = ParVal, Labels=Labels, Weights= Weights, bs = 100000,
        Name = 'ChPgphigw, (16 BSM, 500k, CD, Seed%d), '%(random_seed), Folder = os.getcwd()+'/TrainedModels/')

####
Anlges at position [3, 5] have been converted to Sin and Cos and put at the last columns of the Data.
####
Initializing Preprocesses Variables
Training epoch 10 (took 44.77 sec, time left 11:17:34.881408 sec) loss 0.21834286
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChPgphigw, (16 BSM, 500k, CD, Seed1093), 10 epoch.pth


/home/chen/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Training epoch 100 (took 406.13 sec, time left 12:16:32.522502 sec) loss 0.14363743
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChPgphigw, (16 BSM, 500k, CD, Seed1093), 100 epoch.pth
Training epoch 500 (took 1834.12 sec, time left 12:02:04.252678 sec) loss 0.13206358
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChPgphigw, (16 BSM, 500k, CD, Seed1093), 500 epoch.pth
Training epoch 1000 (took 2274.93 sec, time left 11:23:14.033868 sec) loss 0.13118175
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChPgphigw, (16 BSM, 500k, CD, Seed1093), 1000 epoch.pth
Training epoch 2000 (took 4488.80 sec, time left 10:02:52.434432 sec) loss 0.13064754
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChPgphigw, (16 BSM, 500k, CD, Seed1093), 2000 epoch.pth
Training epoch 3000 (took 4485.21 sec, time left 8:46:04.232774 sec) loss 0.13046

OurCDModel(
  (LinearLayers): ModuleList(
    (0): Linear(in_features=9, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=1, bias=True)
    (4): Linear(in_features=9, out_features=32, bias=True)
    (5): Linear(in_features=32, out_features=32, bias=True)
    (6): Linear(in_features=32, out_features=32, bias=True)
    (7): Linear(in_features=32, out_features=1, bias=True)
    (8): Linear(in_features=9, out_features=32, bias=True)
    (9): Linear(in_features=32, out_features=32, bias=True)
    (10): Linear(in_features=32, out_features=32, bias=True)
    (11): Linear(in_features=32, out_features=1, bias=True)
    (12): Linear(in_features=9, out_features=32, bias=True)
    (13): Linear(in_features=32, out_features=32, bias=True)
    (14): Linear(in_features=32, out_features=32, bias=True)
    (15): Linear(in_features=32, out_features=1, bias=True)


In [5]:
NumEpochs = int(1e4)

td.Data = td.Data[:, :7]
td.CurateAngles([3, 5])

Data, ParVal, Labels, Weights = td.Data, td.ParVal, td.Labels, td.Weights
Data, ParVal, Labels, Weights = Data.float(), ParVal.float(), Labels.float(), Weights.float()

MD = OurCDModel(NumberOfParameters=2, AR=[9,32,32,32,1])
MD.InitPreprocess(Data, ParVal)

OT = OurTrainer(NumEpochs = NumEpochs)
OT.SetSaveAfterEpochs([10,100,500]+list(range(1000, 11000, 1000)))

random_seed = torch.randint(0, 1339, (1,)).item()
OT.Train(MD, Data = Data, Parameters = ParVal, Labels=Labels, Weights= Weights, bs = 100000,
        Name = 'ChPgphigw, (16 BSM, 500k, CD, Seed%d), '%(random_seed), Folder = os.getcwd()+'/TrainedModels/')

####
Anlges at position [3, 5] have been converted to Sin and Cos and put at the last columns of the Data.
####
Initializing Preprocesses Variables
Training epoch 10 (took 48.71 sec, time left 12:17:13.279256 sec) loss 0.21191290
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChPgphigw, (16 BSM, 500k, CD, Seed1041), 10 epoch.pth


/home/chen/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Training epoch 100 (took 433.53 sec, time left 13:07:44.486103 sec) loss 0.14115070
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChPgphigw, (16 BSM, 500k, CD, Seed1041), 100 epoch.pth
Training epoch 500 (took 1921.98 sec, time left 12:39:44.181551 sec) loss 0.13179493
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChPgphigw, (16 BSM, 500k, CD, Seed1041), 500 epoch.pth
Training epoch 1000 (took 2338.40 sec, time left 11:50:36.165979 sec) loss 0.13101889
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChPgphigw, (16 BSM, 500k, CD, Seed1041), 1000 epoch.pth
Training epoch 2000 (took 4783.45 sec, time left 10:34:40.457311 sec) loss 0.13059381
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChPgphigw, (16 BSM, 500k, CD, Seed1041), 2000 epoch.pth
Training epoch 3000 (took 4777.70 sec, time left 9:15:59.557608 sec) loss 0.13044

OurCDModel(
  (LinearLayers): ModuleList(
    (0): Linear(in_features=9, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=1, bias=True)
    (4): Linear(in_features=9, out_features=32, bias=True)
    (5): Linear(in_features=32, out_features=32, bias=True)
    (6): Linear(in_features=32, out_features=32, bias=True)
    (7): Linear(in_features=32, out_features=1, bias=True)
    (8): Linear(in_features=9, out_features=32, bias=True)
    (9): Linear(in_features=32, out_features=32, bias=True)
    (10): Linear(in_features=32, out_features=32, bias=True)
    (11): Linear(in_features=32, out_features=1, bias=True)
    (12): Linear(in_features=9, out_features=32, bias=True)
    (13): Linear(in_features=32, out_features=32, bias=True)
    (14): Linear(in_features=32, out_features=32, bias=True)
    (15): Linear(in_features=32, out_features=1, bias=True)


In [6]:
DataFolder = '/data3/WZ_new_project/h5/Ideal_Data'

td = OurTrainingData([DataFolder + '/ChM_pt300_sm_1.h5',],
                    [DataFolder + '/ChM_pt300_gphi2e-1_gw1e-1.h5',
                     DataFolder + '/ChM_pt300_gphi5e-1_gw2e-1.h5',
                     DataFolder + '/ChM_pt300_gphim2e-1_gw1e-1.h5',
                     DataFolder + '/ChM_pt300_gphim5e-1_gw2e-1.h5',
                     DataFolder + '/ChM_pt300_gphim5e-1.h5',
                     DataFolder + '/ChM_pt300_gphim2e-1.h5',
                     DataFolder + '/ChM_pt300_gphim5e-2.h5',
                     DataFolder + '/ChM_pt300_gphi5e-1.h5',
                     DataFolder + '/ChM_pt300_gphi2e-1.h5',
                     DataFolder + '/ChM_pt300_gphi5e-2.h5',
                     DataFolder + '/ChM_pt300_gwm5e-2.h5',
                     DataFolder + '/ChM_pt300_gwm1e-1.h5',
                     DataFolder + '/ChM_pt300_gwm2e-1.h5',                     
                     DataFolder + '/ChM_pt300_gw5e-2.h5',
                     DataFolder + '/ChM_pt300_gw1e-1.h5',
                     DataFolder + '/ChM_pt300_gw2e-1.h5'],
                     process = 'W-Z', parameters =['Gphi[TeV**-2]', 'GW[TeV**-2]'], 
                     SMNLimits=int(3e6),
                     BSMNLimits=int(5e5))

Loading Data Files for Process: W-Z, with new physics Parameters: ['Gphi[TeV**-2]', 'GW[TeV**-2]']
Only 1D Implemented in Training !

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChM_pt300_gphi2e-1_gw1e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {0.2, 0.1}[TeV**-2] data, Ideal Events. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ}, weight}.
Converted from /data3/WZ_new_project/dat/Ideal_Events/ChM_pt300_gphi2e-1_gw1e-1.dat.gz
Charge = -1 --- Process = W-Z
#####

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChM_pt300_gphi5e-1_gw2e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {0.5, 0.2}[TeV**-2] data, Ideal Events. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ}, weight}.
Converted from /data3/WZ_new_project/dat/Ideal_Events/ChM_pt300_gphi5e-1_gw2e-1.dat.gz
Charge = -1 --- Process = W-Z
#####

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChM_pt300_gphim2e-1_gw1e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {-0.2, 0.1}[TeV**-2] data, Ide

In [8]:
NumEpochs = int(1e4)

td.Data = td.Data[:, :7]
td.CurateAngles([3, 5])

Data, ParVal, Labels, Weights = td.Data, td.ParVal, td.Labels, td.Weights
Data, ParVal, Labels, Weights = Data.float(), ParVal.float(), Labels.float(), Weights.float()

MD = OurCDModel(NumberOfParameters=2, AR=[9,32,32,32,1])
MD.InitPreprocess(Data, ParVal)

OT = OurTrainer(NumEpochs = NumEpochs)
OT.SetSaveAfterEpochs([10,100,500]+list(range(1000, 11000, 1000)))

OT.Train(MD, Data = Data, Parameters = ParVal, Labels=Labels, Weights= Weights, bs = 100000,
        Name = 'ChMgphigw, (16 BSM, 500k, CD, Seed%d), '%(random_seed), Folder = os.getcwd()+'/TrainedModels/')

####
Anlges at position [3, 5] have been converted to Sin and Cos and put at the last columns of the Data.
####
Initializing Preprocesses Variables
Training epoch 10 (took 44.86 sec, time left 11:18:55.435618 sec) loss 0.20842935
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChMgphigw, (16 BSM, 500k, CD, Seed1093), 10 epoch.pth
Training epoch 100 (took 397.04 sec, time left 12:01:50.257040 sec) loss 0.14733735
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChMgphigw, (16 BSM, 500k, CD, Seed1093), 100 epoch.pth
Training epoch 500 (took 1759.09 sec, time left 11:35:31.022031 sec) loss 0.13845392
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChMgphigw, (16 BSM, 500k, CD, Seed1093), 500 epoch.pth
Training epoch 1000 (took 2204.84 sec, time left 11:00:08.518789 sec) loss 0.13774949
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedMo

OurCDModel(
  (LinearLayers): ModuleList(
    (0): Linear(in_features=9, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=1, bias=True)
    (4): Linear(in_features=9, out_features=32, bias=True)
    (5): Linear(in_features=32, out_features=32, bias=True)
    (6): Linear(in_features=32, out_features=32, bias=True)
    (7): Linear(in_features=32, out_features=1, bias=True)
    (8): Linear(in_features=9, out_features=32, bias=True)
    (9): Linear(in_features=32, out_features=32, bias=True)
    (10): Linear(in_features=32, out_features=32, bias=True)
    (11): Linear(in_features=32, out_features=1, bias=True)
    (12): Linear(in_features=9, out_features=32, bias=True)
    (13): Linear(in_features=32, out_features=32, bias=True)
    (14): Linear(in_features=32, out_features=32, bias=True)
    (15): Linear(in_features=32, out_features=1, bias=True)


In [7]:
NumEpochs = int(1e4)

td.Data = td.Data[:, :7]
td.CurateAngles([3, 5])

Data, ParVal, Labels, Weights = td.Data, td.ParVal, td.Labels, td.Weights
Data, ParVal, Labels, Weights = Data.float(), ParVal.float(), Labels.float(), Weights.float()

MD = OurCDModel(NumberOfParameters=2, AR=[9,32,32,32,1])
MD.InitPreprocess(Data, ParVal)

OT = OurTrainer(NumEpochs = NumEpochs)
OT.SetSaveAfterEpochs([10,100,500]+list(range(1000, 11000, 1000)))

OT.Train(MD, Data = Data, Parameters = ParVal, Labels=Labels, Weights= Weights, bs = 100000,
        Name = 'ChMgphigw, (16 BSM, 500k, CD, Seed%d), '%(random_seed), Folder = os.getcwd()+'/TrainedModels/')

####
Anlges at position [3, 5] have been converted to Sin and Cos and put at the last columns of the Data.
####
Initializing Preprocesses Variables
Training epoch 10 (took 47.38 sec, time left 11:57:05.774109 sec) loss 0.21161512
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChMgphigw, (16 BSM, 500k, CD, Seed1041), 10 epoch.pth
Training epoch 100 (took 425.32 sec, time left 12:52:09.616413 sec) loss 0.14970854
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChMgphigw, (16 BSM, 500k, CD, Seed1041), 100 epoch.pth
Training epoch 500 (took 1897.46 sec, time left 12:28:58.492749 sec) loss 0.13875407
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChMgphigw, (16 BSM, 500k, CD, Seed1041), 500 epoch.pth
Training epoch 1000 (took 2365.52 sec, time left 11:49:33.843709 sec) loss 0.13790245
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedMo

OurCDModel(
  (LinearLayers): ModuleList(
    (0): Linear(in_features=9, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=1, bias=True)
    (4): Linear(in_features=9, out_features=32, bias=True)
    (5): Linear(in_features=32, out_features=32, bias=True)
    (6): Linear(in_features=32, out_features=32, bias=True)
    (7): Linear(in_features=32, out_features=1, bias=True)
    (8): Linear(in_features=9, out_features=32, bias=True)
    (9): Linear(in_features=32, out_features=32, bias=True)
    (10): Linear(in_features=32, out_features=32, bias=True)
    (11): Linear(in_features=32, out_features=1, bias=True)
    (12): Linear(in_features=9, out_features=32, bias=True)
    (13): Linear(in_features=32, out_features=32, bias=True)
    (14): Linear(in_features=32, out_features=32, bias=True)
    (15): Linear(in_features=32, out_features=1, bias=True)


## balanced SM/BSM ratio

added more SM points

In [9]:
DataFolder = '/data3/WZ_new_project/h5/Ideal_Data'

td = OurTrainingData([DataFolder + '/ChP_pt300_sm_1.h5',
                     DataFolder + '/ChP_pt300_sm_2.h5',
                     DataFolder + '/ChP_pt300_sm_3.h5'],
                    [DataFolder + '/ChP_pt300_gphi2e-1_gw1e-1.h5',
                     DataFolder + '/ChP_pt300_gphi5e-1_gw2e-1.h5',
                     DataFolder + '/ChP_pt300_gphim2e-1_gw1e-1.h5',
                     DataFolder + '/ChP_pt300_gphim5e-1_gw2e-1.h5',
                     DataFolder + '/ChP_pt300_gphim5e-1.h5',
                     DataFolder + '/ChP_pt300_gphim2e-1.h5',
                     DataFolder + '/ChP_pt300_gphim5e-2.h5',
                     DataFolder + '/ChP_pt300_gphi5e-1.h5',
                     DataFolder + '/ChP_pt300_gphi2e-1.h5',
                     DataFolder + '/ChP_pt300_gphi5e-2.h5',
                     DataFolder + '/ChP_pt300_gwm5e-2.h5',
                     DataFolder + '/ChP_pt300_gwm1e-1.h5',
                     DataFolder + '/ChP_pt300_gwm2e-1.h5',                     
                     DataFolder + '/ChP_pt300_gw5e-2.h5',
                     DataFolder + '/ChP_pt300_gw1e-1.h5',
                     DataFolder + '/ChP_pt300_gw2e-1.h5'],
                     process = 'W+Z', parameters =['Gphi[TeV**-2]', 'GW[TeV**-2]'], 
                     SMNLimits=int(3e6),
                     BSMNLimits=int(5e5))

NumEpochs = int(1e4)

td.Data = td.Data[:, :7]
td.CurateAngles([3, 5])

Data, ParVal, Labels, Weights = td.Data, td.ParVal, td.Labels, td.Weights
Data, ParVal, Labels, Weights = Data.float(), ParVal.float(), Labels.float(), Weights.float()

MD = OurCDModel(NumberOfParameters=2, AR=[9,32,32,32,1])
MD.InitPreprocess(Data, ParVal)

OT = OurTrainer(NumEpochs = NumEpochs)
OT.SetSaveAfterEpochs([10,100,500]+list(range(1000, 11000, 1000)))

random_seed = torch.randint(0, 1339, (1,)).item()
OT.Train(MD, Data = Data, Parameters = ParVal, Labels=Labels, Weights= Weights, bs = 100000,
        Name = 'ChPgphigw, (16 BSM, 500k, CD, Seed%d), '%(random_seed), Folder = os.getcwd()+'/TrainedModels/')

Loading Data Files for Process: W+Z, with new physics Parameters: ['Gphi[TeV**-2]', 'GW[TeV**-2]']
Only 1D Implemented in Training !

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChP_pt300_gphi2e-1_gw1e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {0.2, 0.1}[TeV**-2] data, Ideal Events. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ}, weight}.
Converted from /data3/WZ_new_project/dat/Ideal_Events/ChP_pt300_gphi2e-1_gw1e-1.dat.gz
Charge = 1 --- Process = W+Z
#####

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChP_pt300_gphi5e-1_gw2e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {0.5, 0.2}[TeV**-2] data, Ideal Events. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ}, weight}.
Converted from /data3/WZ_new_project/dat/Ideal_Events/ChP_pt300_gphi5e-1_gw2e-1.dat.gz
Charge = 1 --- Process = W+Z
#####

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChP_pt300_gphim2e-1_gw1e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {-0.2, 0.1}[TeV**-2] data, Ideal

####
Anlges at position [3, 5] have been converted to Sin and Cos and put at the last columns of the Data.
####
Initializing Preprocesses Variables
Training epoch 10 (took 54.42 sec, time left 13:43:40.183148 sec) loss 0.22133550
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChPgphigw, (16 BSM, 500k, CD, Seed687), 10 epoch.pth
Training epoch 100 (took 489.29 sec, time left 14:48:09.354231 sec) loss 0.14446956
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChPgphigw, (16 BSM, 500k, CD, Seed687), 100 epoch.pth
Training epoch 500 (took 2166.39 sec, time left 14:16:23.798849 sec) loss 0.13236594
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChPgphigw, (16 BSM, 500k, CD, Seed687), 500 epoch.pth
Training epoch 1000 (took 2706.17 sec, time left 13:31:32.411013 sec) loss 0.13123044
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModel

OurCDModel(
  (LinearLayers): ModuleList(
    (0): Linear(in_features=9, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=1, bias=True)
    (4): Linear(in_features=9, out_features=32, bias=True)
    (5): Linear(in_features=32, out_features=32, bias=True)
    (6): Linear(in_features=32, out_features=32, bias=True)
    (7): Linear(in_features=32, out_features=1, bias=True)
    (8): Linear(in_features=9, out_features=32, bias=True)
    (9): Linear(in_features=32, out_features=32, bias=True)
    (10): Linear(in_features=32, out_features=32, bias=True)
    (11): Linear(in_features=32, out_features=1, bias=True)
    (12): Linear(in_features=9, out_features=32, bias=True)
    (13): Linear(in_features=32, out_features=32, bias=True)
    (14): Linear(in_features=32, out_features=32, bias=True)
    (15): Linear(in_features=32, out_features=1, bias=True)


In [11]:
DataFolder = '/data3/WZ_new_project/h5/Ideal_Data'

td = OurTrainingData([DataFolder + '/ChM_pt300_sm_1.h5',
                     DataFolder + '/ChM_pt300_sm_2.h5',
                     DataFolder + '/ChM_pt300_sm_3.h5'],
                    [DataFolder + '/ChM_pt300_gphi2e-1_gw1e-1.h5',
                     DataFolder + '/ChM_pt300_gphi5e-1_gw2e-1.h5',
                     DataFolder + '/ChM_pt300_gphim2e-1_gw1e-1.h5',
                     DataFolder + '/ChM_pt300_gphim5e-1_gw2e-1.h5',
                     DataFolder + '/ChM_pt300_gphim5e-1.h5',
                     DataFolder + '/ChM_pt300_gphim2e-1.h5',
                     DataFolder + '/ChM_pt300_gphim5e-2.h5',
                     DataFolder + '/ChM_pt300_gphi5e-1.h5',
                     DataFolder + '/ChM_pt300_gphi2e-1.h5',
                     DataFolder + '/ChM_pt300_gphi5e-2.h5',
                     DataFolder + '/ChM_pt300_gwm5e-2.h5',
                     DataFolder + '/ChM_pt300_gwm1e-1.h5',
                     DataFolder + '/ChM_pt300_gwm2e-1.h5',                     
                     DataFolder + '/ChM_pt300_gw5e-2.h5',
                     DataFolder + '/ChM_pt300_gw1e-1.h5',
                     DataFolder + '/ChM_pt300_gw2e-1.h5'],
                     process = 'W-Z', parameters =['Gphi[TeV**-2]', 'GW[TeV**-2]'], 
                     SMNLimits=int(3e6),
                     BSMNLimits=int(5e5))

NumEpochs = int(1e4)

td.Data = td.Data[:, :7]
td.CurateAngles([3, 5])

Data, ParVal, Labels, Weights = td.Data, td.ParVal, td.Labels, td.Weights
Data, ParVal, Labels, Weights = Data.float(), ParVal.float(), Labels.float(), Weights.float()

MD = OurCDModel(NumberOfParameters=2, AR=[9,32,32,32,1])
MD.InitPreprocess(Data, ParVal)

OT = OurTrainer(NumEpochs = NumEpochs)
OT.SetSaveAfterEpochs([10,100,500]+list(range(1000, 11000, 1000)))

OT.Train(MD, Data = Data, Parameters = ParVal, Labels=Labels, Weights= Weights, bs = 100000,
        Name = 'ChMgphigw, (16 BSM, 500k, CD, Seed%d), '%(random_seed), Folder = os.getcwd()+'/TrainedModels/')

Loading Data Files for Process: W-Z, with new physics Parameters: ['Gphi[TeV**-2]', 'GW[TeV**-2]']
Only 1D Implemented in Training !

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChM_pt300_gphi2e-1_gw1e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {0.2, 0.1}[TeV**-2] data, Ideal Events. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ}, weight}.
Converted from /data3/WZ_new_project/dat/Ideal_Events/ChM_pt300_gphi2e-1_gw1e-1.dat.gz
Charge = -1 --- Process = W-Z
#####

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChM_pt300_gphi5e-1_gw2e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {0.5, 0.2}[TeV**-2] data, Ideal Events. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ}, weight}.
Converted from /data3/WZ_new_project/dat/Ideal_Events/ChM_pt300_gphi5e-1_gw2e-1.dat.gz
Charge = -1 --- Process = W-Z
#####

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChM_pt300_gphim2e-1_gw1e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {-0.2, 0.1}[TeV**-2] data, Ide

####
Anlges at position [3, 5] have been converted to Sin and Cos and put at the last columns of the Data.
####
Initializing Preprocesses Variables
Training epoch 10 (took 52.87 sec, time left 13:20:10.250760 sec) loss 0.19306214
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChMgphigw, (16 BSM, 500k, CD, Seed687), 10 epoch.pth
Training epoch 100 (took 482.54 sec, time left 14:34:35.794174 sec) loss 0.14669555
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChMgphigw, (16 BSM, 500k, CD, Seed687), 100 epoch.pth
Training epoch 500 (took 2160.22 sec, time left 14:11:49.425822 sec) loss 0.13833730
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChMgphigw, (16 BSM, 500k, CD, Seed687), 500 epoch.pth
Training epoch 1000 (took 2698.38 sec, time left 13:28:12.285856 sec) loss 0.13770500
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModel

OurCDModel(
  (LinearLayers): ModuleList(
    (0): Linear(in_features=9, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=1, bias=True)
    (4): Linear(in_features=9, out_features=32, bias=True)
    (5): Linear(in_features=32, out_features=32, bias=True)
    (6): Linear(in_features=32, out_features=32, bias=True)
    (7): Linear(in_features=32, out_features=1, bias=True)
    (8): Linear(in_features=9, out_features=32, bias=True)
    (9): Linear(in_features=32, out_features=32, bias=True)
    (10): Linear(in_features=32, out_features=32, bias=True)
    (11): Linear(in_features=32, out_features=1, bias=True)
    (12): Linear(in_features=9, out_features=32, bias=True)
    (13): Linear(in_features=32, out_features=32, bias=True)
    (14): Linear(in_features=32, out_features=32, bias=True)
    (15): Linear(in_features=32, out_features=1, bias=True)


In [8]:
DataFolder = '/data3/WZ_new_project/h5/Ideal_Data'

td = OurTrainingData([DataFolder + '/ChP_pt300_sm_1.h5',
                     DataFolder + '/ChP_pt300_sm_2.h5',
                     DataFolder + '/ChP_pt300_sm_3.h5'],
                    [DataFolder + '/ChP_pt300_gphi2e-1_gw1e-1.h5',
                     DataFolder + '/ChP_pt300_gphi5e-1_gw2e-1.h5',
                     DataFolder + '/ChP_pt300_gphim2e-1_gw1e-1.h5',
                     DataFolder + '/ChP_pt300_gphim5e-1_gw2e-1.h5',
                     DataFolder + '/ChP_pt300_gphim5e-1.h5',
                     DataFolder + '/ChP_pt300_gphim2e-1.h5',
                     DataFolder + '/ChP_pt300_gphim5e-2.h5',
                     DataFolder + '/ChP_pt300_gphi5e-1.h5',
                     DataFolder + '/ChP_pt300_gphi2e-1.h5',
                     DataFolder + '/ChP_pt300_gphi5e-2.h5',
                     DataFolder + '/ChP_pt300_gwm5e-2.h5',
                     DataFolder + '/ChP_pt300_gwm1e-1.h5',
                     DataFolder + '/ChP_pt300_gwm2e-1.h5',                     
                     DataFolder + '/ChP_pt300_gw5e-2.h5',
                     DataFolder + '/ChP_pt300_gw1e-1.h5',
                     DataFolder + '/ChP_pt300_gw2e-1.h5'],
                     process = 'W+Z', parameters =['Gphi[TeV**-2]', 'GW[TeV**-2]'], 
                     SMNLimits=int(3e6),
                     BSMNLimits=int(5e5))

NumEpochs = int(1e4)

td.Data = td.Data[:, :7]
td.CurateAngles([3, 5])

Data, ParVal, Labels, Weights = td.Data, td.ParVal, td.Labels, td.Weights
Data, ParVal, Labels, Weights = Data.float(), ParVal.float(), Labels.float(), Weights.float()

MD = OurCDModel(NumberOfParameters=2, AR=[9,32,32,32,1])
MD.InitPreprocess(Data, ParVal)

OT = OurTrainer(NumEpochs = NumEpochs)
OT.SetSaveAfterEpochs([10,100,500]+list(range(1000, 11000, 1000)))

random_seed = torch.randint(0, 1339, (1,)).item()
OT.Train(MD, Data = Data, Parameters = ParVal, Labels=Labels, Weights= Weights, bs = 100000,
        Name = 'ChPgphigw, (16 BSM, 500k, CD, Seed%d), '%(random_seed), Folder = os.getcwd()+'/TrainedModels/')

Loading Data Files for Process: W+Z, with new physics Parameters: ['Gphi[TeV**-2]', 'GW[TeV**-2]']
Only 1D Implemented in Training !

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChP_pt300_gphi2e-1_gw1e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {0.2, 0.1}[TeV**-2] data, Ideal Events. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ}, weight}.
Converted from /data3/WZ_new_project/dat/Ideal_Events/ChP_pt300_gphi2e-1_gw1e-1.dat.gz
Charge = 1 --- Process = W+Z
#####

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChP_pt300_gphi5e-1_gw2e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {0.5, 0.2}[TeV**-2] data, Ideal Events. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ}, weight}.
Converted from /data3/WZ_new_project/dat/Ideal_Events/ChP_pt300_gphi5e-1_gw2e-1.dat.gz
Charge = 1 --- Process = W+Z
#####

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChP_pt300_gphim2e-1_gw1e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {-0.2, 0.1}[TeV**-2] data, Ideal

####
Anlges at position [3, 5] have been converted to Sin and Cos and put at the last columns of the Data.
####
Initializing Preprocesses Variables
Training epoch 10 (took 56.71 sec, time left 14:18:20.776720 sec) loss 0.24936733
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChPgphigw, (16 BSM, 500k, CD, Seed1103), 10 epoch.pth
Training epoch 100 (took 512.39 sec, time left 15:29:37.585714 sec) loss 0.14258671
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChPgphigw, (16 BSM, 500k, CD, Seed1103), 100 epoch.pth
Training epoch 500 (took 2282.65 sec, time left 15:01:09.463137 sec) loss 0.13195612
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChPgphigw, (16 BSM, 500k, CD, Seed1103), 500 epoch.pth
Training epoch 1000 (took 2851.74 sec, time left 14:14:34.434798 sec) loss 0.13106100
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedMo

OurCDModel(
  (LinearLayers): ModuleList(
    (0): Linear(in_features=9, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=1, bias=True)
    (4): Linear(in_features=9, out_features=32, bias=True)
    (5): Linear(in_features=32, out_features=32, bias=True)
    (6): Linear(in_features=32, out_features=32, bias=True)
    (7): Linear(in_features=32, out_features=1, bias=True)
    (8): Linear(in_features=9, out_features=32, bias=True)
    (9): Linear(in_features=32, out_features=32, bias=True)
    (10): Linear(in_features=32, out_features=32, bias=True)
    (11): Linear(in_features=32, out_features=1, bias=True)
    (12): Linear(in_features=9, out_features=32, bias=True)
    (13): Linear(in_features=32, out_features=32, bias=True)
    (14): Linear(in_features=32, out_features=32, bias=True)
    (15): Linear(in_features=32, out_features=1, bias=True)


In [9]:
DataFolder = '/data3/WZ_new_project/h5/Ideal_Data'

td = OurTrainingData([DataFolder + '/ChM_pt300_sm_1.h5',
                     DataFolder + '/ChM_pt300_sm_2.h5',
                     DataFolder + '/ChM_pt300_sm_3.h5'],
                    [DataFolder + '/ChM_pt300_gphi2e-1_gw1e-1.h5',
                     DataFolder + '/ChM_pt300_gphi5e-1_gw2e-1.h5',
                     DataFolder + '/ChM_pt300_gphim2e-1_gw1e-1.h5',
                     DataFolder + '/ChM_pt300_gphim5e-1_gw2e-1.h5',
                     DataFolder + '/ChM_pt300_gphim5e-1.h5',
                     DataFolder + '/ChM_pt300_gphim2e-1.h5',
                     DataFolder + '/ChM_pt300_gphim5e-2.h5',
                     DataFolder + '/ChM_pt300_gphi5e-1.h5',
                     DataFolder + '/ChM_pt300_gphi2e-1.h5',
                     DataFolder + '/ChM_pt300_gphi5e-2.h5',
                     DataFolder + '/ChM_pt300_gwm5e-2.h5',
                     DataFolder + '/ChM_pt300_gwm1e-1.h5',
                     DataFolder + '/ChM_pt300_gwm2e-1.h5',                     
                     DataFolder + '/ChM_pt300_gw5e-2.h5',
                     DataFolder + '/ChM_pt300_gw1e-1.h5',
                     DataFolder + '/ChM_pt300_gw2e-1.h5'],
                     process = 'W-Z', parameters =['Gphi[TeV**-2]', 'GW[TeV**-2]'], 
                     SMNLimits=int(3e6),
                     BSMNLimits=int(5e5))

NumEpochs = int(1e4)

td.Data = td.Data[:, :7]
td.CurateAngles([3, 5])

Data, ParVal, Labels, Weights = td.Data, td.ParVal, td.Labels, td.Weights
Data, ParVal, Labels, Weights = Data.float(), ParVal.float(), Labels.float(), Weights.float()

MD = OurCDModel(NumberOfParameters=2, AR=[9,32,32,32,1])
MD.InitPreprocess(Data, ParVal)

OT = OurTrainer(NumEpochs = NumEpochs)
OT.SetSaveAfterEpochs([10,100,500]+list(range(1000, 11000, 1000)))

OT.Train(MD, Data = Data, Parameters = ParVal, Labels=Labels, Weights= Weights, bs = 100000,
        Name = 'ChMgphigw, (16 BSM, 500k, CD, Seed%d), '%(random_seed), Folder = os.getcwd()+'/TrainedModels/')

Loading Data Files for Process: W-Z, with new physics Parameters: ['Gphi[TeV**-2]', 'GW[TeV**-2]']
Only 1D Implemented in Training !

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChM_pt300_gphi2e-1_gw1e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {0.2, 0.1}[TeV**-2] data, Ideal Events. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ}, weight}.
Converted from /data3/WZ_new_project/dat/Ideal_Events/ChM_pt300_gphi2e-1_gw1e-1.dat.gz
Charge = -1 --- Process = W-Z
#####

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChM_pt300_gphi5e-1_gw2e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {0.5, 0.2}[TeV**-2] data, Ideal Events. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ}, weight}.
Converted from /data3/WZ_new_project/dat/Ideal_Events/ChM_pt300_gphi5e-1_gw2e-1.dat.gz
Charge = -1 --- Process = W-Z
#####

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChM_pt300_gphim2e-1_gw1e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {-0.2, 0.1}[TeV**-2] data, Ide

####
Anlges at position [3, 5] have been converted to Sin and Cos and put at the last columns of the Data.
####
Initializing Preprocesses Variables
Training epoch 10 (took 57.37 sec, time left 14:28:13.274567 sec) loss 0.22166409
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChMgphigw, (16 BSM, 500k, CD, Seed1103), 10 epoch.pth
Training epoch 100 (took 516.74 sec, time left 15:37:48.492124 sec) loss 0.14922570
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChMgphigw, (16 BSM, 500k, CD, Seed1103), 100 epoch.pth
Training epoch 500 (took 2281.83 sec, time left 15:02:28.981852 sec) loss 0.13890547
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChMgphigw, (16 BSM, 500k, CD, Seed1103), 500 epoch.pth
Training epoch 1000 (took 2856.90 sec, time left 14:15:58.577525 sec) loss 0.13804360
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedMo

OurCDModel(
  (LinearLayers): ModuleList(
    (0): Linear(in_features=9, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=1, bias=True)
    (4): Linear(in_features=9, out_features=32, bias=True)
    (5): Linear(in_features=32, out_features=32, bias=True)
    (6): Linear(in_features=32, out_features=32, bias=True)
    (7): Linear(in_features=32, out_features=1, bias=True)
    (8): Linear(in_features=9, out_features=32, bias=True)
    (9): Linear(in_features=32, out_features=32, bias=True)
    (10): Linear(in_features=32, out_features=32, bias=True)
    (11): Linear(in_features=32, out_features=1, bias=True)
    (12): Linear(in_features=9, out_features=32, bias=True)
    (13): Linear(in_features=32, out_features=32, bias=True)
    (14): Linear(in_features=32, out_features=32, bias=True)
    (15): Linear(in_features=32, out_features=1, bias=True)


In [10]:
DataFolder = '/data3/WZ_new_project/h5/Ideal_Data'

td = OurTrainingData([DataFolder + '/ChP_pt300_sm_1.h5',
                     DataFolder + '/ChP_pt300_sm_2.h5',
                     DataFolder + '/ChP_pt300_sm_3.h5'],
                    [DataFolder + '/ChP_pt300_gphi2e-1_gw1e-1.h5',
                     DataFolder + '/ChP_pt300_gphi5e-1_gw2e-1.h5',
                     DataFolder + '/ChP_pt300_gphim2e-1_gw1e-1.h5',
                     DataFolder + '/ChP_pt300_gphim5e-1_gw2e-1.h5',
                     DataFolder + '/ChP_pt300_gphim5e-1.h5',
                     DataFolder + '/ChP_pt300_gphim2e-1.h5',
                     DataFolder + '/ChP_pt300_gphim5e-2.h5',
                     DataFolder + '/ChP_pt300_gphi5e-1.h5',
                     DataFolder + '/ChP_pt300_gphi2e-1.h5',
                     DataFolder + '/ChP_pt300_gphi5e-2.h5',
                     DataFolder + '/ChP_pt300_gwm5e-2.h5',
                     DataFolder + '/ChP_pt300_gwm1e-1.h5',
                     DataFolder + '/ChP_pt300_gwm2e-1.h5',                     
                     DataFolder + '/ChP_pt300_gw5e-2.h5',
                     DataFolder + '/ChP_pt300_gw1e-1.h5',
                     DataFolder + '/ChP_pt300_gw2e-1.h5'],
                     process = 'W+Z', parameters =['Gphi[TeV**-2]', 'GW[TeV**-2]'], 
                     SMNLimits=int(3e6),
                     BSMNLimits=int(5e5))

NumEpochs = int(1e4)

td.Data = td.Data[:, :7]
td.CurateAngles([3, 5])

Data, ParVal, Labels, Weights = td.Data, td.ParVal, td.Labels, td.Weights
Data, ParVal, Labels, Weights = Data.float(), ParVal.float(), Labels.float(), Weights.float()

MD = OurCDModel(NumberOfParameters=2, AR=[9,32,32,32,1])
MD.InitPreprocess(Data, ParVal)

OT = OurTrainer(NumEpochs = NumEpochs)
OT.SetSaveAfterEpochs([10,100,500]+list(range(1000, 11000, 1000)))

random_seed = torch.randint(0, 1339, (1,)).item()
OT.Train(MD, Data = Data, Parameters = ParVal, Labels=Labels, Weights= Weights, bs = 100000,
        Name = 'ChPgphigw, (16 BSM, 500k, CD, Seed%d), '%(random_seed), Folder = os.getcwd()+'/TrainedModels/')

Loading Data Files for Process: W+Z, with new physics Parameters: ['Gphi[TeV**-2]', 'GW[TeV**-2]']
Only 1D Implemented in Training !

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChP_pt300_gphi2e-1_gw1e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {0.2, 0.1}[TeV**-2] data, Ideal Events. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ}, weight}.
Converted from /data3/WZ_new_project/dat/Ideal_Events/ChP_pt300_gphi2e-1_gw1e-1.dat.gz
Charge = 1 --- Process = W+Z
#####

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChP_pt300_gphi5e-1_gw2e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {0.5, 0.2}[TeV**-2] data, Ideal Events. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ}, weight}.
Converted from /data3/WZ_new_project/dat/Ideal_Events/ChP_pt300_gphi5e-1_gw2e-1.dat.gz
Charge = 1 --- Process = W+Z
#####

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChP_pt300_gphim2e-1_gw1e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {-0.2, 0.1}[TeV**-2] data, Ideal

####
Anlges at position [3, 5] have been converted to Sin and Cos and put at the last columns of the Data.
####
Initializing Preprocesses Variables
Training epoch 10 (took 55.55 sec, time left 14:00:46.060543 sec) loss 0.20816803
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChPgphigw, (16 BSM, 500k, CD, Seed929), 10 epoch.pth
Training epoch 100 (took 499.01 sec, time left 15:05:52.886008 sec) loss 0.14137304
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChPgphigw, (16 BSM, 500k, CD, Seed929), 100 epoch.pth
Training epoch 500 (took 2219.09 sec, time left 14:36:28.810875 sec) loss 0.13186528
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChPgphigw, (16 BSM, 500k, CD, Seed929), 500 epoch.pth
Training epoch 1000 (took 2773.62 sec, time left 13:51:10.063738 sec) loss 0.13099979
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModel

OurCDModel(
  (LinearLayers): ModuleList(
    (0): Linear(in_features=9, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=1, bias=True)
    (4): Linear(in_features=9, out_features=32, bias=True)
    (5): Linear(in_features=32, out_features=32, bias=True)
    (6): Linear(in_features=32, out_features=32, bias=True)
    (7): Linear(in_features=32, out_features=1, bias=True)
    (8): Linear(in_features=9, out_features=32, bias=True)
    (9): Linear(in_features=32, out_features=32, bias=True)
    (10): Linear(in_features=32, out_features=32, bias=True)
    (11): Linear(in_features=32, out_features=1, bias=True)
    (12): Linear(in_features=9, out_features=32, bias=True)
    (13): Linear(in_features=32, out_features=32, bias=True)
    (14): Linear(in_features=32, out_features=32, bias=True)
    (15): Linear(in_features=32, out_features=1, bias=True)


In [11]:
DataFolder = '/data3/WZ_new_project/h5/Ideal_Data'

td = OurTrainingData([DataFolder + '/ChM_pt300_sm_1.h5',
                     DataFolder + '/ChM_pt300_sm_2.h5',
                     DataFolder + '/ChM_pt300_sm_3.h5'],
                    [DataFolder + '/ChM_pt300_gphi2e-1_gw1e-1.h5',
                     DataFolder + '/ChM_pt300_gphi5e-1_gw2e-1.h5',
                     DataFolder + '/ChM_pt300_gphim2e-1_gw1e-1.h5',
                     DataFolder + '/ChM_pt300_gphim5e-1_gw2e-1.h5',
                     DataFolder + '/ChM_pt300_gphim5e-1.h5',
                     DataFolder + '/ChM_pt300_gphim2e-1.h5',
                     DataFolder + '/ChM_pt300_gphim5e-2.h5',
                     DataFolder + '/ChM_pt300_gphi5e-1.h5',
                     DataFolder + '/ChM_pt300_gphi2e-1.h5',
                     DataFolder + '/ChM_pt300_gphi5e-2.h5',
                     DataFolder + '/ChM_pt300_gwm5e-2.h5',
                     DataFolder + '/ChM_pt300_gwm1e-1.h5',
                     DataFolder + '/ChM_pt300_gwm2e-1.h5',                     
                     DataFolder + '/ChM_pt300_gw5e-2.h5',
                     DataFolder + '/ChM_pt300_gw1e-1.h5',
                     DataFolder + '/ChM_pt300_gw2e-1.h5'],
                     process = 'W-Z', parameters =['Gphi[TeV**-2]', 'GW[TeV**-2]'], 
                     SMNLimits=int(3e6),
                     BSMNLimits=int(5e5))

NumEpochs = int(1e4)

td.Data = td.Data[:, :7]
td.CurateAngles([3, 5])

Data, ParVal, Labels, Weights = td.Data, td.ParVal, td.Labels, td.Weights
Data, ParVal, Labels, Weights = Data.float(), ParVal.float(), Labels.float(), Weights.float()

MD = OurCDModel(NumberOfParameters=2, AR=[9,32,32,32,1])
MD.InitPreprocess(Data, ParVal)

OT = OurTrainer(NumEpochs = NumEpochs)
OT.SetSaveAfterEpochs([10,100,500]+list(range(1000, 11000, 1000)))

OT.Train(MD, Data = Data, Parameters = ParVal, Labels=Labels, Weights= Weights, bs = 100000,
        Name = 'ChMgphigw, (16 BSM, 500k, CD, Seed%d), '%(random_seed), Folder = os.getcwd()+'/TrainedModels/')

Loading Data Files for Process: W-Z, with new physics Parameters: ['Gphi[TeV**-2]', 'GW[TeV**-2]']
Only 1D Implemented in Training !

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChM_pt300_gphi2e-1_gw1e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {0.2, 0.1}[TeV**-2] data, Ideal Events. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ}, weight}.
Converted from /data3/WZ_new_project/dat/Ideal_Events/ChM_pt300_gphi2e-1_gw1e-1.dat.gz
Charge = -1 --- Process = W-Z
#####

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChM_pt300_gphi5e-1_gw2e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {0.5, 0.2}[TeV**-2] data, Ideal Events. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ}, weight}.
Converted from /data3/WZ_new_project/dat/Ideal_Events/ChM_pt300_gphi5e-1_gw2e-1.dat.gz
Charge = -1 --- Process = W-Z
#####

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChM_pt300_gphim2e-1_gw1e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {-0.2, 0.1}[TeV**-2] data, Ide

####
Anlges at position [3, 5] have been converted to Sin and Cos and put at the last columns of the Data.
####
Initializing Preprocesses Variables
Training epoch 10 (took 55.53 sec, time left 14:00:23.763699 sec) loss 0.21146284
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChMgphigw, (16 BSM, 500k, CD, Seed929), 10 epoch.pth
Training epoch 100 (took 500.43 sec, time left 15:08:08.924804 sec) loss 0.14930682
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChMgphigw, (16 BSM, 500k, CD, Seed929), 100 epoch.pth
Training epoch 500 (took 2227.77 sec, time left 14:39:39.659810 sec) loss 0.13862538
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModels/ChMgphigw, (16 BSM, 500k, CD, Seed929), 500 epoch.pth
Training epoch 1000 (took 2783.77 sec, time left 13:54:11.815259 sec) loss 0.13782436
Model successfully saved.
Path: /home/chen/Documents/2DQuadraticClassifier/TrainedModel

OurCDModel(
  (LinearLayers): ModuleList(
    (0): Linear(in_features=9, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=1, bias=True)
    (4): Linear(in_features=9, out_features=32, bias=True)
    (5): Linear(in_features=32, out_features=32, bias=True)
    (6): Linear(in_features=32, out_features=32, bias=True)
    (7): Linear(in_features=32, out_features=1, bias=True)
    (8): Linear(in_features=9, out_features=32, bias=True)
    (9): Linear(in_features=32, out_features=32, bias=True)
    (10): Linear(in_features=32, out_features=32, bias=True)
    (11): Linear(in_features=32, out_features=1, bias=True)
    (12): Linear(in_features=9, out_features=32, bias=True)
    (13): Linear(in_features=32, out_features=32, bias=True)
    (14): Linear(in_features=32, out_features=32, bias=True)
    (15): Linear(in_features=32, out_features=1, bias=True)
